# ADM-HW4

## 2. Alphabetical Sort

In this section are contained the discussions regarding time complexity about the functions created for the resolution of the exercise.

### Counting sort

In [1]:
def counting_sort(A):
    """
    This function takes in input a list of integers and return a new ordered list, based on the first one.
    :param A: the list to order
    :type A: list of int
    :return: ordered list
    :rtype: list of int
    """
    k = max(A)  # O(n)
    n = len(A)  # O(1)
    B = [0 for i in range(n)]  # O(n)
    C = [0 for i in range(k+1)]  # O(k+1) -> Theta(n)

    for i in range(n):  # O(n)
        C[A[i]] += 1  # O(1)
        # each element C[i] is equal to the number of i in the array A

    for i in range(1, k+1):  # O(k+1) -> Theta(n)
        C[i] += C[i-1]  # O(1)
        # each C[i] is now equal to the number of elements in A equal or less than i

    for i in range(n-1, -1, -1):  # O(n)
        el = A[i]  # O(1)
        index = C[el]-1  # O(1)
        B[index] = el  # O(1)  # insert A[i] in the right position in B, given by C[i]-1
        C[el] += -1  # O(1)  # decrease by one the number of elements equal or less than A[i]

    return B

After analyzing the **counting_sort** function it is easy to calculate its time complexity, since it corresponds to a sum of complexities not bigger than **_O(n)_**. So, assuming O(k) = Theta(n), the total time complexity of the algorithm is **_O(n)_**.